In [1]:
import pandas as pd
import numpy as np
import warnings

import seaborn as sns
import matplotlib.pyplot as plt

#Plotly Libraris
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
import plotly.io as pio



warnings.filterwarnings('ignore')

pio.templates.default = "ggplot2"

#dataframe display settings
pd.set_option('display.max_columns', 5000000)
pd.set_option('display.max_rows', 50000000)

In [2]:
## Data from dataset 3
industry_trends = pd.read_csv('data/dataset3.csv')
industry_trends.rename(columns={"Unnamed: 0": "StartupID", "Unnamed: 6": "City_ZIP"}, inplace=True)
## Data from dataset 4
startup_info = pd.read_csv('data/dataset4.csv')


In [3]:
startup_info['founded_year'] = pd.to_datetime(startup_info['founded_at'], errors = 'coerce').dt.year
startup_info['funding_total_usd'].replace('-', 0, inplace=True)
startup_info['funding_total_usd'] = startup_info['funding_total_usd'].astype(float)
startup_info = startup_info.drop_duplicates()

# Cleanning dataset
clean_startup_info = startup_info.drop(['permalink', 'homepage_url'], axis=1)

# Remove the NaN name
clean_startup_info.dropna(how='any', subset=['name'], axis=0, inplace=True)

#Extracting year value from "first_funding_at" and changing to int
clean_startup_info['first_funding_at'] = clean_startup_info.first_funding_at.str.split("-").str[0]
clean_startup_info['first_funding_at'] = clean_startup_info['first_funding_at'].astype(float)
#Extracting year value from "last_funding_at" and changing to int
clean_startup_info['last_funding_at'] = clean_startup_info.last_funding_at.str.split("-").str[0]
clean_startup_info['last_funding_at'] = clean_startup_info['last_funding_at'].astype(float)

# Treat Nan category
clean_startup_info['category_list'] = clean_startup_info['category_list'].replace(np.nan, 'other')
clean_startup_info.head()

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,founded_year
0,#fame,Media,10000000.0,operating,IND,16,Mumbai,Mumbai,1,NaN,2015.0,2015.0,NaN
1,:Qounter,Application Platforms|Real Time|Social Network...,700000.0,operating,USA,DE,DE - Other,Delaware City,2,2014-09-04,2014.0,2014.0,2014.0
2,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,3406878.0,operating,NaN,NaN,NaN,NaN,1,NaN,2014.0,2014.0,NaN
3,0-6.com,Curated Web,2000000.0,operating,CHN,22,Beijing,Beijing,1,2007-01-01,2008.0,2008.0,2007.0
4,004 Technologies,Software,0.0,operating,USA,IL,"Springfield, Illinois",Champaign,1,2010-01-01,2014.0,2014.0,2010.0


In [4]:
corr_startups = clean_startup_info[clean_startup_info['status'] != 'ipo']
corr_startups.reset_index(drop=True, inplace=True)
corr_startups['is_successful'] = corr_startups['status'].apply(lambda x: 1 if x in ['operating', 'acquired'] else 0)

corr_startups.columns


Index(['name', 'category_list', 'funding_total_usd', 'status', 'country_code',
       'state_code', 'region', 'city', 'funding_rounds', 'founded_at',
       'first_funding_at', 'last_funding_at', 'founded_year', 'is_successful'],
      dtype='object')

In [5]:
# Split the 'category_list' column by '|' and create dummy variables
categories_dummy = corr_startups['category_list'].str.get_dummies(sep='|')


In [6]:
# One-hot encode 'country_code' and 'city' columns
country_dummy = pd.get_dummies(corr_startups['country_code'], prefix='country')

# Concatenate the dummy variables with the DataFrame
encoded_corr = pd.concat([corr_startups, categories_dummy, country_dummy], axis=1)
encoded_corr.head(3)


,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,founded_year,is_successful,3D,3D Printing,3D Technology,Accounting,Active Lifestyle,Ad Targeting,Adaptive Equipment,Advanced Materials,Adventure Travel,Advertising,Advertising Exchanges,Advertising Networks,Advertising Platforms,Advice,Aerospace,Agriculture,Air Pollution Control,Algorithms,All Markets,All Students,Alternative Medicine,Alumni,Analytics,Android,Angels,Animal Feed,Anything Capital Intensive,App Discovery,App Marketing,App Stores,Application Performance Monitoring,Application Platforms,Apps,Aquaculture,Architecture,Archiving,Art,Artificial Intelligence,Artists Globally,Assisitive Technology,Assisted Living,Auctions,Audio,Audiobooks,Augmented Reality,Auto,Automated Kiosk,Automotive,B2B,B2B Express Delivery,BPO Services,Babies,Baby Accessories,Baby Boomers,Baby Safety,Banking,Batteries,Beauty,Bicycles,Big Data,Big Data Analytics,Billing,Bio-Pharm,Biofuels,Bioinformatics,Biomass Power Generation,Biometrics,Biotechnology,Biotechnology and Semiconductor,Bitcoin,Blogging Platforms,Boating Industry,Brand Marketing,Brewing,Bridging Online and Offline,Broadcasting,Brokers,Browser Extensions,Building Owners,Building Products,Business Analytics,Business Development,Business Information Systems,Business Intelligence,Business Productivity,Business Services,Business Travelers,CAD,CRM,Cable,Call Center Automation,Cannabis,Carbon,Career Management,Career Planning,Cars,Casual Games,Cause Marketing,Celebrity,Certification Test,Charities,Charity,Charter Schools,Chat,Chemicals,Child Care,China Internet,Civil Engineers,Classifieds,Clean Energy,Clean Technology,Clean Technology IT,Clinical Trials,Cloud Computing,Cloud Data Services,Cloud Gaming,Cloud Infrastructure,Cloud Management,Cloud Security,Cloud-Based Music,Coffee,Collaboration,Collaborative Consumption,Collectibles,College Campuses,College Recruiting,Colleges,Comics,Commercial Real Estate,Commercial Solar,Commodities,Communications Hardware,Communications Infrastructure,Communities,Comparison Shopping,Computer Vision,Computers,Concentrated Solar Power,Concerts,Console Gaming,Construction,Consulting,Consumer Behavior,Consumer Electronics,Consumer Goods,Consumer Internet,Consumer Lending,Consumers,Contact Centers,Contact Management,Content,Content Creators,Content Delivery,Content Discovery,Content Syndication,Contests,Cooking,Corporate IT,Corporate Training,Corporate Wellness,Cosmetic Surgery,Cosmetics,Coupons,Coworking,Craft Beer,Creative,Creative Industries,Credit,Credit Cards,Crowdfunding,Crowdsourcing,Cryptocurrency,Curated Web,Custom Retail,Customer Service,Customer Support Tools,Cyber,Cyber Security,DIY,DOD/Military,Darknet,Data Center Automation,Data Center Infrastructure,Data Centers,Data Integration,Data Mining,Data Privacy,Data Security,Data Visualization,Databases,Debt Collecting,Deep Information Technology,Defense,Delivery,Demographies,Dental,Design,Designers,Developer APIs,Developer Tools,Development Platforms,Diabetes,Diagnostics,Dietary Supplements,Digital Entertainment,Digital Media,Digital Rights Management,Digital Signage,Direct Advertising,Direct Marketing,Direct Sales,Discounts,Displays,Disruptive Models,Distribution,Distributors,Diving,Doctors,Document Management,Domains,Drones,E-Books,E-Commerce,E-Commerce Platforms,EBooks,EDA Tools,Early Stage IT,Early-Stage Technology,East Africa,EdTech,Ediscovery,Education,Educational Games,Edutainment,Elder Care,Elderly,Electric Vehicles,Electrical Distribution,Electronic Health Records,Electronics,Email,Email Marketing,Email Newsletters,Embedded Hardware and Software,Emerging Markets,Employer Benefits Programs,Employment,Energy,Energy Efficiency,Energy IT,Energy Management,Energy Storage,Engineering Firms,English-Speaking,Enterprise 2.0,Enterprise Application,Enterprise Hardware,Enterprise Purchasing,Enterprise Resource Planning,Enterprise Search,Enterprise Security,Enterprise Soft

In [2]:
# Calculate the correlation matrix
correlation_matrix = encoded_corr.corr()

# Plot the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix[['is_successful']].sort_values(by='is_successful', ascending=False),
            annot=True, cmap='coolwarm', fmt=".2f", annot_kws={"size": 10})
plt.title('Correlation with is_successful')
plt.show()


NameError: name 'encoded_corr' is not defined

In [1]:
correlation_matrix[['is_successful']].sort_values(by='is_successful', ascending=False)

NameError: name 'correlation_matrix' is not defined